## Report3

**Date:** 2024-05-26  
**Author:** Piotr Szepietowski

## External Python Libraries Used

    - Numpy
    - matplotlib
    - scipy

## Task 1
The task is to implement the power method to determine the condition number of a matrix. It should be a function that takes a matrix and returns:

    - the computed condition number
    - the condition number from the built-in function
    - the difference between the methods

## Implementation of the Power Method

The method has been implemented as the function `power_method_cond()`, which takes a matrix $A$ and optional parameters `tol` (corresponding to the threshold value determining convergence) and `max_iter` (specifying the maximum number of iterations).  
It returns the result for the power method, the built-in function, and the difference between the results.  
When calculating eigenvalues, it uses the Rayleigh quotient.  
The `solve` function from `scipy` (solving systems of equations) is used to implement the inverse power method to improve calculations. Thanks to these improvements, it was also possible to minimize problems with operations for non-diagonalized matrices.

---

### Description of the power method for calculating the condition number of a matrix

The condition number of a matrix $A$ (in the 2-norm) is defined as the ratio of the largest to the smallest singular value of the matrix:

$$
\kappa(A) = \frac{\sigma_{\max}(A)}{\sigma_{\min}(A)}
$$

where $\sigma_{\max}(A)$ and $\sigma_{\min}(A)$ are the largest and smallest singular values of the matrix $A$, respectively.

#### Power method

1. **Largest eigenvalue**  
    The power method allows approximating the largest eigenvalue of the matrix $A^T A$, which equals the square of the largest singular value $\sigma_{\max}^2$.  
    We iteratively determine the vector $b_k$ and the eigenvalue $\lambda_{\max}$:
    $$
    b_{k+1} = \frac{A^T A b_k}{\|A^T A b_k\|}
    $$
    $$
    \lambda_{\max} = b_k^T (A^T A b_k)
    $$
    Then $\sigma_{\max} = \sqrt{\lambda_{\max}}$.

2. **Smallest eigenvalue (inverse power method)**  
    We determine the smallest eigenvalue $\lambda_{\min}$ by applying the inverse power method, i.e., solving the system $A^T A y = c_k$ in each iteration.  
    $\sigma_{\min} = \sqrt{\lambda_{\min}}$.

3. **Calculation of the condition number**  
    Finally:
    $$
    \kappa(A) = \frac{\sigma_{\max}}{\sigma_{\min}}
    $$

#### Advantages and limitations

- The power method is simple and effective for large, sparse matrices.
- The accuracy of determining the smallest eigenvalue may be limited, especially for matrices with very large condition numbers or close to singularity.
- In practice, for matrices with very small or very large elements, the method may be less accurate than built-in numerical algorithms.

---

**Summary:**  
The power method allows for approximate determination of the condition number of a matrix by iteratively estimating the largest and smallest eigenvalues of the matrix $A^T A$, which corresponds to the largest and smallest singular values of the matrix $A$.

In [ ]:
import numpy as np
from scipy.linalg import solve

def power_method_cond(A, tol=1e-10, max_iter=1000):
    n = A.shape[0]
    
    try:
        test_vec = np.ones(n)
        solve(A, test_vec)
    except np.linalg.LinAlgError:
        return np.inf, np.inf, 0.0
    
    ATA = A.T @ A
    b_k = np.random.rand(n)
    b_k = b_k / np.linalg.norm(b_k)
    
    lambda_max_prev = 0
    for _ in range(max_iter):
        Ab_k = ATA @ b_k
        lambda_max = np.dot(b_k, Ab_k)
        b_k_next = Ab_k / np.linalg.norm(Ab_k)
        
        if abs(lambda_max - lambda_max_prev) < tol:
            break
            
        lambda_max_prev = lambda_max
        b_k = b_k_next
    
    sigma_max = np.sqrt(lambda_max)
    
    c_k = np.random.rand(n)
    c_k = c_k / np.linalg.norm(c_k)
    
    lambda_min_prev = np.inf
    for _ in range(max_iter):
        try:
            y = solve(ATA, c_k)
        except np.linalg.LinAlgError:
            return np.inf, np.inf, 0.0
            
        lambda_min_inv = np.dot(c_k, y)
        
        if lambda_min_inv == 0:
            return np.inf, np.inf, 0.0
            
        lambda_min = 1.0 / lambda_min_inv
        c_k_next = y / np.linalg.norm(y)
        if abs(lambda_min - lambda_min_prev) < tol:
            break
            
        lambda_min_prev = lambda_min
        c_k = c_k_next
    
    sigma_min = np.sqrt(lambda_min)
    cond_power = sigma_max / sigma_min
    cond_builtin = np.linalg.cond(A)
    diff = abs(cond_power - cond_builtin)
    
    return cond_power, cond_builtin, diff

A = np.array([[2, 1], [1, 3]])
cond_power, cond_builtin, diff = power_method_cond(A)
print("Condition number (power method):", cond_power)
print("Condition number (built-in function):", cond_builtin)
print("Difference:", diff)

A = np.random.rand(10, 10)
cond_power, cond_builtin, diff = power_method_cond(A)
print("Condition number (power method):", cond_power)
print("Condition number (built-in function):", cond_builtin)
print("Difference:", diff)

## Testing methods on the given C matrix
$$
C = \begin{bmatrix}
4 & 2 & -5 & 2 \\
1 & 5 & 3 & 9 \\
2 & 2 & 5 & -7 \\
1 & 4 & -1 & 1 \\
\end{bmatrix}
$$

In [ ]:
C = np.array([[4,2,-5,2], [1,5,3,9], [2,2,5,-7],[1,4,-1,1]])

cond_power, cond_builtin, diff = power_method_cond(C)
print("Condition number (power method):", cond_power)
print("Condition number (built-in function):", cond_builtin)
print("Difference:", diff)

## Task 2 
Using the created function, investigate the influence of matrix size and type on the method's efficiency and the obtained condition numbers.

## Testing methods on random matrices of increasing size
The methods are called on random matrices of increasing sizes to examine the results depending on matrix size.

In [ ]:
import matplotlib.pyplot as plt

sizes = [x for x in range(2, 21, 2)] + [x for x in range(30, 101, 10)] + [x for x in range(150, 1001, 50)] + [x for x in range(1001, 3001, 100)]
cond_powers = []
cond_builtins = []
diffs = []

for n in sizes:
    A_rand = np.random.rand(n, n)
    cond_power, cond_builtin, diff = power_method_cond(A_rand)
    cond_powers.append(cond_power)
    cond_builtins.append(cond_builtin)
    diffs.append(diff)

fig, axes = plt.subplots(1, 2, figsize=(18, 6))

axes[0].plot(sizes, cond_powers, marker='o', label='Power method')
axes[0].plot(sizes, cond_builtins, marker='x', label='Built-in function')
axes[0].set_xlabel('Matrix size')
axes[0].set_ylabel('Condition number')
axes[0].set_title('Comparison of condition number')
axes[0].legend()
axes[0].grid(True)

axes[1].plot(sizes, diffs, marker='s', color='red')
axes[1].set_xlabel('Matrix size')
axes[1].set_ylabel('Difference')
axes[1].set_title('Difference between methods')
axes[1].grid(True)

plt.tight_layout()
plt.show()

## Conclusions
With the given implementation, the results of both methods are quite similar.
The graph of the difference between method results closely corresponds to peaks in both methods' results, suggesting a decrease in calculation accuracy for larger coefficients.

In [ ]:
# Plot of proportion of difference to coefficient value (power method)
proportion = [d / cp if cp != 0 else 0 for d, cp in zip(diffs, cond_powers)]

plt.figure(figsize=(10, 6))
plt.plot(sizes, proportion, marker='o')
plt.xlabel('Matrix size')
plt.ylabel('Difference / Coefficient (power method)')
plt.title('Proportion of difference to condition number value')
plt.grid(True)
plt.show()

Apart from a significant increase in the difference relative to the coefficient value at the very end of the range, the graph confirms the existence of a greater difference for coefficients with higher values.

## Investigation of methods based on matrix type

## Hilbert matrix and near-singular matrix

In [ ]:
# Hilbert matrix 4x4
H = np.array([[1/(i+j+1) for j in range(4)] for i in range(4)])
cond_power, cond_builtin, diff = power_method_cond(H)
print("Hilbert 4x4:")
print("Condition number (power method):", cond_power)
print("Condition number (built-in function):", cond_builtin)
print("Difference:", diff)

# Near-singular matrix (e.g., two very similar columns)
B = np.array([[1, 1, 1], [2, 2.000001, 2], [3, 3, 3.000001]])
cond_power, cond_builtin, diff = power_method_cond(B)
print("\nNear-singular 3x3:")
print("Condition number (power method):", cond_power)
print("Condition number (built-in function):", cond_builtin)
print("Difference:", diff)

The method returns very accurate results for the Hilbert matrix, while the result for the near-singular matrix differs significantly.

## Matrices with different numerical scales

In [ ]:
# Matrix with very small elements (order of 1e-10)
small_scale = np.array([[1e-10, 2e-10], [3e-10, 4e-10]])
cond_power_small, cond_builtin_small, diff_small = power_method_cond(small_scale)
print("Very small values (1e-10):")
print("Condition number (power method):", cond_power_small)
print("Condition number (built-in function):", cond_builtin_small)
print("Difference:", diff_small)

# Matrix with very large elements (order of 1e+10)
large_scale = np.array([[1e+10, 2e+10], [3e+10, 4e+10]])
cond_power_large, cond_builtin_large, diff_large = power_method_cond(large_scale)
print("\nVery large values (1e+10):")
print("Condition number (power method):", cond_power_large)
print("Condition number (built-in function):", cond_builtin_large)
print("Difference:", diff_large)

For small values, the differences are significantly larger than for a very similar matrix with large elements.